In [270]:
#imports
import re

# Task 1 - Table Driven Lexer

In [271]:
# TOKEN CLASS
class Token:
    def __init__(self, inputValue, inputType):
        self.value = inputValue
        self.type = inputType


# LEXER CLASS
class Lexer:
    def __init__(self, input):
        self.code = input
        self.tokenList = []
        self.pos = 0
        self.tokenTable = [
            (r"not|let|return|if|else|for|while|fun|->", "KEYWORDS"),

            (r"\b(float|int|bool|colour)\b", "TYPE"),
            
            (r"\b(true|false)\b", "BOOLEANLITERAL"),
            (r"\d+\.\d+", "FLOATLITERAL"),
            (r"\d+", "INTEGERLITERAL"),
            (r"\#[0-9a-fA-F]{6}", "COLOURLITERAL"),

            (r"\b(__width)\b", "PADWIDTH"),
            (r"\b(__height)\b", "PADHEIGHT"),

            (r"\b(__read)\b", "__READ"),
            (r"\b(__randi)\b", "__RANDI"),

            (r"[\*/]|and|AND", "MULTIPLICATIVEOP"),
            (r"[\+]|or|OR", "ADDITIVEOP"),
            (r"==|!=|<=|>=|[\<\>]", "RELATIONALOP"),

            (r"[a-zA-Z]([a-zA-Z0-9]|_)*", "IDENTIFIER"),

            (r"\b(__print)\b", "__PRINT"),
            (r"\b(__delay)\b", "__DELAY"),
            (r"\b(__pixelr)\b", "__PIXELR"),
            (r"\b(__pixel)\b", "__PIXEL"),

            (r"[\,\(\)\-\=\:\;\{\}]", "OTHER"),
        ]
        
        print("L E X E R")
        self.scanText()
        # print("\nToken List:")
        # # print(self.tokenList)
        print("Code accepted by Lexer")

    # SCAN GIVEN CODE
    def scanText(self):
        # loops while less than size of string
        while self.pos < len(self.code):
            # ignores whitespace
            if self.code[self.pos] == " " or self.code[self.pos] == "\n":
                self.pos += 1
                
            # if not whitespace
            else:
                value, type = None, None

                # goes through table of patterns
                for pattern, tokenType in self.tokenTable:
                    match = re.match(pattern, self.code[self.pos:])

                    # if they match
                    if match is not None:
                        type = tokenType
                        value = match.group()
                        break
                
                # if value and type remained empyty
                if value == None or type == None:
                    raise Exception("Error in Syntax")
                # if both value and type have been assigned
                else:
                    if type == "KEYWORDS" or type == "OTHER":
                        self.tokenList.append((value.upper(), value))
                        # self.tokenList.append(Token(value, "<"+value+">"))
                    else:
                        self.tokenList.append((type, value))
                        # self.tokenList.append(Token(value, type))
                    self.pos += len(value)



# Task 2 - Hand-crafted LL(k) parser

In [272]:
# TERMINAL NODE CLASS
class TerminalNode:
    def __init__(self, name, value):
        self.name = name
        self.value = value


# NON-TERMINAL NODE CLASS
class NTerminalNode:
    def __init__(self, name):
        self.name = name
        self.children = []
        

# PARSER CLASS
class Parser:
    def __init__(self, tokens):

        self.tokenList = tokens
        self.tree = None

        print("\nP A R S E R")
        self.parseCode()
        print("Code Accepted by Parser")        

    # PARSE GIVEN CODE
    def parseCode(self):
        self.program()
        
    # GET NEXT TOKEN FROM LEXER
    def getNextToken(self):
        self.tokenList.pop(0)

    # FOR EACH RULE
    def padRead(self, tree):
        # creates a new PADREAD node
        nTree = NTerminalNode("PADREAD")
        nTree.children.append(TerminalNode("__READ", self.tokenList[0][1])); self.getNextToken()
        self.expr(nTree)

        # checks if the next token is a comma
        if self.tokenList[0][0] == ",":
            self.getNextToken()
            self.expr(nTree)
            tree.children.append(nTree)            
        else:
            raise Exception ("Error: missing ','")

    def padRandI(self, tree):
        nTree = NTerminalNode("PADRANDI")
        nTree.children.append(TerminalNode("__RANDI", self.tokenList[0][1])); self.getNextToken()
        self.expr(nTree)
        tree.children.append(nTree)

    def literal(self, tree):
        nTree = NTerminalNode("LITERAL")
        nTree.children.append(TerminalNode(self.tokenList[0][0], self.tokenList[0][1])); self.getNextToken()
        tree.children.append(nTree)

    def actualParams(self, tree):
        nTree = NTerminalNode("ACTUALPARAMS")
        self.expr(nTree)

        # loops till it reaches the end of the COMMAN, EXPR pairs
        while self.tokenList and self.tokenList[0][0] == ",":
            self.getNextToken()
            self.expr(nTree)
        
        tree.children.append(nTree)

    def functionCall(self, tree):
        nTree = NTerminalNode("FUNCTIONCALL")
        nTree.children.append(TerminalNode("IDENTIFIER", self.tokenList[0][1])); self.getNextToken()
        self.getNextToken()
    
        # if there are parameters
        if self.tokenList[0][0] != ")":
            self.actualParams(nTree)

            if self.tokenList[0][0] == ")":
                self.getNextToken()
                tree.children.append(nTree)
            else:
                raise Exception ("Error: missing ')'")
                
        # if there are no parameters
        elif self.tokenList[0][0] == ")":
            self.getNextToken()
            tree.children.append(nTree)     

    def subExpr(self, tree):
        nTree = NTerminalNode("SUBEXPR")
        self.getNextToken()
        self.expr(nTree)

        if self.tokenList[0][0] == ")":
            self.getNextToken()
            tree.children.append(nTree)
        else:
            raise Exception ("Error: missing ')'")

    def unary(self, tree):
        nTree = NTerminalNode("UNARY")
        nTree.children.append(TerminalNode(self.tokenList[0][0], self.tokenList[0][1])); self.getNextToken()
        self.expr(nTree)
        tree.children.append(nTree)

    def factor(self, tree):
        nTree = NTerminalNode("FACTOR")

        # LITERAL
        if self.tokenList[0][0] in ["BOOLEANLITERAL", "INTEGERLITERAL", "FLOATLITERAL", "COLOURLITERAL"]:
            self.literal(nTree)

        # IDENTIFIER & FUNCTIONCALL
        elif self.tokenList[0][0] == "IDENTIFIER":   
            if len(self.tokenList) > 1:
                if self.tokenList[1][0] == "(":
                    self.functionCall(nTree)
                else:
                    nTree.children.append(TerminalNode("IDENTIFIER", self.tokenList[0][1])); self.getNextToken()
            else:
                nTree.children.append(TerminalNode("IDENTIFIER", self.tokenList[0][1])); self.getNextToken()

        # SUBEXPR
        elif self.tokenList[0][0] == "(":
            self.subExpr(nTree)

        # UNARY
        elif self.tokenList[0][0] == "-" or self.tokenList[0][0] == "NOT":
            self.unary(nTree)

        # PADRANDI
        elif self.tokenList[0][0] == "__RANDI":
            self.padRandI(nTree)

        # PADWIDTH
        elif self.tokenList[0][0] == "PADWIDTH":
            nTree.children.append(TerminalNode("PADWIDTH", self.tokenList[0][1])); self.getNextToken()

        # PADHEIGHT
        elif self.tokenList[0][0] == "PADHEIGHT":
            nTree.children.append(TerminalNode("PADHEIGHT", self.tokenList[0][1])); self.getNextToken()

        #PADREAD
        elif self.tokenList[0][0] == "__READ":
            self.padRead(nTree)

        else:
            raise Exception ("Error: factor not recognised")
        
        tree.children.append(nTree)
    
    def term(self, tree):
        nTree = NTerminalNode("TERM")
        self.factor(nTree)
        
        while self.tokenList and self.tokenList[0][0] == "MULTIPLICATIVEOP":
            nTree.children.append(TerminalNode("MULTIPLICATIVEOP", self.tokenList[0][1])); self.getNextToken()
            self.factor(nTree)
        
        tree.children.append(nTree)
        
    def simpleExpr(self, tree):
        nTree = NTerminalNode("SIMPLEEXPR")
        self.term(nTree)
        
        while self.tokenList and self.tokenList[0][0] == "ADDITIVEOP" or self.tokenList[0][1] == "-":
            nTree.children.append(TerminalNode("ADDITIVEOP", self.tokenList[0][1])); self.getNextToken()
            self.term(nTree)
        
        tree.children.append(nTree)

    def expr(self, tree):
        nTree = NTerminalNode("EXPR")
        self.simpleExpr(nTree)
        
        while self.tokenList and self.tokenList[0][0] == "RELATIONALOP":
            nTree.children.append(TerminalNode("RELATIONALOP", self.tokenList[0][1])); self.getNextToken()
            self.simpleExpr(nTree)

        tree.children.append(nTree)

    def assignment(self, tree):
        nTree = NTerminalNode("ASSIGNMENT")

        if self.tokenList[0][0] == "IDENTIFIER":
            nTree.children.append(TerminalNode("IDENTIFIER", self.tokenList[0][1])); self.getNextToken()

            if self.tokenList[0][0] == "=":
                nTree.children.append(TerminalNode("EQUALS", self.tokenList[0][1])); self.getNextToken()

                self.expr(nTree)
                tree.children.append(nTree)
            
            else:
                raise Exception ("Error: missing '='")
        else:
            raise Exception ("Error: missing IDENTIFIER")

    def variableDecl(self, tree):
        nTree = NTerminalNode("VARIABLEDECL")

        if self.tokenList[0][0] == "LET":
            self.getNextToken()
            
            if self.tokenList[0][0] == "IDENTIFIER":
                nTree.children.append(TerminalNode("IDENTIFIER", self.tokenList[0][1])); self.getNextToken()
                
                if self.tokenList[0][0] == ":":
                    self.getNextToken()

                    if self.tokenList[0][0] == "TYPE":
                        nTree.children.append(TerminalNode("TYPE", self.tokenList[0][1])); self.getNextToken()

                        if self.tokenList[0][0] == "=":
                            nTree.children.append(TerminalNode("EQUALS", self.tokenList[0][1])); self.getNextToken()
                            self.expr(nTree)
                            tree.children.append(nTree)

                        else:
                            raise Exception ("Error: missing '='")
                    else:
                        raise Exception ("Error: missing TYPE")
                else:
                    raise Exception ("Error: missing ':'")
            else:
                raise Exception ("Error: missing 'IDENTIFIER'")
        else:
            raise Exception ("Error: missing LET keyword")

    def printStatement(self, tree):
        nTree = NTerminalNode("PRINTSTATEMENT")

        if self.tokenList[0][0] == "__PRINT":
            self.getNextToken()            
            self.expr(nTree)
            tree.children.append(nTree)
        
        else:
            raise Exception ("Error in Parser")

    def delayStatement(self, tree):
        nTree = NTerminalNode("DELAYSTATEMENT")

        if self.tokenList[0][0] == "__DELAY":
            self.getNextToken()            
            self.expr(nTree)
            tree.children.append(nTree)

        else:
            raise Exception ("Error in Parser")

    def pixelStatement(self, tree):
        nTree = NTerminalNode("PIXELSTATEMENT")

        if self.tokenList[0][0] == "__PIXELR":
            nTree.children.append(TerminalNode("__PIXELR", self.tokenList[0][1])); self.getNextToken()
            self.expr(nTree)

            if self.tokenList[0][0] == ",":
                self.getNextToken()
                self.expr(nTree)

                if self.tokenList[0][0] == ",":
                    self.getNextToken()
                    self.expr(nTree)

                    if self.tokenList[0][0] == ",":
                        self.getNextToken()
                        self.expr(nTree)

                        if self.tokenList[0][0] == ",":
                            self.getNextToken()
                            self.expr(nTree)

                            tree.children.append(nTree)
                        
                        else:
                            raise Exception ("Error in Parser")
                    else:
                        raise Exception ("Error in Parser")
                else:
                    raise Exception ("Error in Parser")
            else:
                raise Exception ("Error in Parser")
        
        elif self.tokenList[0][0] == "__PIXEL":
            nTree.children.append(TerminalNode("__PIXEL", self.tokenList[0][1])); self.getNextToken()
            self.expr(nTree)

            if self.tokenList[0][0] == ",":
                self.getNextToken()
                self.expr(nTree)

                if self.tokenList[0][0] == ",":
                    self.getNextToken()
                    self.expr(nTree)

                    tree.children.append(nTree)
                
                else:
                    raise Exception ("Error in Parser")
            else:
                raise Exception ("Error in Parser")
        else:
            raise Exception ("Error in Parser")

    def rtrnStatement(self, tree):
        nTree = NTerminalNode("RTRNSTATEMENT")

        if self.tokenList[0][0] == "RETURN":
            self.getNextToken()
            self.expr(nTree)
            tree.children.append(nTree)

    def ifStatement(self, tree):
        nTree = NTerminalNode("IFSTATEMENT")

        if self.tokenList[0][0] == "IF":
            self.getNextToken()

            if self.tokenList[0][0] == "(":
                self.getNextToken()
                self.expr(nTree)
                
                if self.tokenList[0][0] == ")":
                    self.getNextToken()
                    self.block(nTree)
                    
                    if self.tokenList and self.tokenList[0][0] == "ELSE":
                        self.getNextToken()
                        self.block(nTree)
                        tree.children.append(nTree)

                    else:
                        tree.children.append(nTree)
                
                else:
                    raise Exception ("Error: missing ')'")
            else:
                raise Exception ("Error: missing '('")
        else:
            raise Exception ("Error: missing IF keyword")                

    def forStatement(self, tree):
        nTree = NTerminalNode("FORSTATEMENT")

        if self.tokenList[0][0] == "FOR":
            self.getNextToken()

            if self.tokenList[0][0] == "(":
                self.getNextToken()

                if self.tokenList[0][0] == "LET":
                    self.variableDecl(nTree)

                    if self.tokenList[0][0] == ";":
                        self.getNextToken()
                        self.expr(nTree)
                        self.getNextToken()

                        if self.tokenList[0][0] == "IDENTIFIER":
                            self.assignment(nTree)
                            
                            if self.tokenList[0][0] == ")":
                                self.getNextToken()
                                self.block(nTree)
                                tree.children.append(nTree)
                            
                            else:
                                raise Exception ("Error: missing ')'")
                            
                        elif self.tokenList[0][0] == ")":
                            self.getNextToken()
                            self.block(nTree)
                            tree.children.append(nTree)
                        
                        else:
                            raise Exception ("Error: missing ')'")
                    else:
                        raise Exception ("Error: missing ';'")
                        
                elif self.tokenList[0][0] == ";":
                    self.getNextToken()
                    self.expr(nTree)
                    self.getNextToken()

                    if self.tokenList[0][0] == "IDENTIFIER":
                        self.assignment(nTree)
                        
                        if self.tokenList[0][0] == ")":
                            self.getNextToken()
                            self.block(nTree)
                            tree.children.append(nTree)
                        
                        else:
                            raise Exception ("Error: missing ')")
                        
                    elif self.tokenList[0][0] == ")":
                        self.getNextToken()
                        self.block(nTree)
                        tree.children.append(nTree)
                    
                    else:
                        raise Exception ("Error: missing ')'")
                else:
                    raise Exception ("Error: missing LET keyword")
            else:
                raise Exception ("Error: missing '('")
        else:
            raise Exception ("Error: missing FOR keyword")

    def whileStatement(self, tree):
        nTree = NTerminalNode("WHILESTATEMENT")
        self.getNextToken()

        if self.tokenList[0][0] == "(":
            self.getNextToken()
            self.expr(nTree)

            if self.tokenList[0][0] == ")":
                self.getNextToken()
                self.block(nTree)
                tree.children.append(nTree)
            
            else:
                raise Exception ("Error: missing ')'")
        
        else:
            raise Exception ("Error: missing '('")

    def formalParam(self, tree):
        nTree = NTerminalNode("FORMALPARAM")

        if self.tokenList[0][0] == "IDENTIFIER":
            nTree.children.append(TerminalNode("IDENTIFIER", self.tokenList[0][1])); self.getNextToken()

            if self.tokenList[0][0] == ":":
                self.getNextToken()

                if self.tokenList[0][0] == "TYPE":
                    nTree.children.append(TerminalNode("TYPE", self.tokenList[0][1])); self.getNextToken()
                    
                    tree.children.append(nTree)
                
                else:
                    raise Exception ("Error: missing TYPE")
            
            else:
                raise Exception ("Error: missin ':")
        
        else:
            raise Exception ("Error: missing parameter")

    def formalParams(self, tree):
        nTree = NTerminalNode("FORMALPARAMS")

        self.formalParam(nTree)

        while self.tokenList and self.tokenList[0][0] == ",":
            self.getNextToken()
            self.formalParam(nTree)

        tree.children.append(nTree)

    def functionDecl(self,tree):
        nTree = NTerminalNode("FUNCTIONDECL")
        self.getNextToken()
        
        if self.tokenList[0][0] == "IDENTIFIER":
            nTree.children.append(TerminalNode("IDENTIFIER", self.tokenList[0][1])); self.getNextToken()

            if self.tokenList[0][0] == "(":
                self.getNextToken()

                # if the function has parameters
                if self.tokenList[0][0] == "IDENTIFIER":
                    self.formalParams(nTree)

                    if self.tokenList[0][0] == ")":
                        self.getNextToken()
                            
                        if self.tokenList[0][0] == "->":
                            nTree.children.append(TerminalNode("ARROW", self.tokenList[0][1])); self.getNextToken()

                            if self.tokenList[0][0] == "TYPE":
                                nTree.children.append(TerminalNode("TYPE", self.tokenList[0][1])); self.getNextToken()
                                self.block(nTree)
                                tree.children.append(nTree)
                            
                            else:
                                raise Exception ("Error: invalid type")
                            
                        else:
                            raise Exception ("Error: missing arrow")
                        
                    else:
                        raise Exception ("Error: missing ')'")      
                
                # if the function doesnt have parameters
                elif self.tokenList[0][0] == ")":
                    self.getNextToken()
                        
                    if self.tokenList[0][0] == "->":
                        nTree.children.append(TerminalNode("ARROW", self.tokenList[0][1])); self.getNextToken()

                        if self.tokenList[0][0] == "TYPE":
                            nTree.children.append(TerminalNode("TYPE", self.tokenList[0][1])); self.getNextToken()
                            self.block(nTree)
                            tree.children.append(nTree)
                        
                        else:
                            raise Exception ("Error: missing type")
                        
                    else:
                        raise Exception ("Error: missing ARROW")
                
                else:
                    raise Exception ("Error: missing PARAMETERS")
            
            else:
                raise Exception ("Error: missing '('")
            
        else:
            raise Exception ("Error: invalid function name")
                            
    def statement(self, tree): 
        nTree = NTerminalNode("STATEMENT")

        # VARIABLEDECL
        if self.tokenList[0][0] == "LET":
            self.variableDecl(nTree)

            if self.tokenList[0][0] == ";":
                self.getNextToken()
            else:
                raise Exception ("Error: missing semi-colon")

        # ASSIGNMENT
        elif self.tokenList[0][0] == "IDENTIFIER":
            self.assignment(nTree)
            
            if self.tokenList and self.tokenList[0][0] == ";":
                self.getNextToken()

            else:
                raise Exception ("Error: missing semi-colon")

        # PRINTSTATEMENT
        elif self.tokenList[0][0] == "__PRINT":
            self.printStatement(nTree)

            if self.tokenList[0][0] == ";":
                self.getNextToken()

            else:
                raise Exception ("Error: missing semi-colon")

        # DELAYSTATEMENT
        elif self.tokenList[0][0] == "__DELAY":
            self.delayStatement(nTree)

            if self.tokenList[0][0] == ";":
                self.getNextToken()

            else:
                raise Exception ("Error: missing semi-colon")

        # PIXELSTATEMENT
        elif self.tokenList[0][0] in ["__PIXEL", "__PIXELR"]:
            self.pixelStatement(nTree)

            if self.tokenList[0][0] == ";":
                self.getNextToken()

            else:
                raise Exception ("Error: missing semi-colon")

        # IFSTATEMENT
        elif self.tokenList[0][0] == "IF":
            self.ifStatement(nTree)

        # FORSTATEMENT
        elif self.tokenList[0][0] == "FOR":
            self.forStatement(nTree)

        # WHILESTATEMENT
        elif self.tokenList[0][0] == "WHILE":
            self.whileStatement(nTree)

        # RETURN
        elif self.tokenList[0][0] == "RETURN":
            self.rtrnStatement(nTree)
            
            if self.tokenList[0][0] == ";":
                self.getNextToken()

            else:
                raise Exception ("Error: missing semi-colon")

        # FUNCTIONDECL
        elif self.tokenList[0][0] == "FUN":
            self.functionDecl(nTree)

        # BLOCK
        elif self.tokenList[0][0] == "{":
            self.block(nTree)
            
        else:
            raise Exception ("Error: invalid STATEMENT")
        
        tree.children.append(nTree)

    def block(self, tree):
        nTree = NTerminalNode("BLOCK")
        self.getNextToken()
        
        # if block isnt empty
        if self.tokenList[0][0] != "}":
            
            # loops while there are still statements in the block
            while self.tokenList[0][0] != "}":
                (self.statement(nTree))

            self.getNextToken()

        # if block is empty
        elif self.tokenList[0][0] == "}":
            self.getNextToken()

        tree.children.append(nTree)

    def program(self):
        tree = NTerminalNode("PROGRAM")
        
        # loops for each statement in the program
        while self.tokenList:
            (self.statement(tree))

        self.tree = tree



# Task 3 - AST XML Generation Pass

In [273]:
# XML CLASS
class XMLGeneration:
    def __init__(self, tree):
        self.AST = tree
        self.stack = []
        self.space = ""

        print("\nX M L   G E N E R A T I O N")
        xmlFull = self.createXML(self.AST)
        xmlMin = self.createXMLMinimized(self.AST)

        self.printXML(xmlFull, xmlMin)
        print("Printed XML Full and XML Minimized")       

    # CREATE FULL XML AND MINIMIZED XML
    def createXML(self, tree):
        temp = []
        if str(type(tree)) == "<class '__main__.NTerminalNode'>":
            temp.append(self.space + "<" + tree.name + ">")

            self.stack.append(tree.name); self.space += "  "

            for child in tree.children:
                temp.append(self.createXML(child))
                temp = self.flatten(temp)
            
            self.space = self.space[:-2]
            temp.append(self.space + "</" + self.stack[-1] + ">")
            
            self.stack.pop(); 
        else:
            temp.append(self.space + "<" + str(tree.name) + "> " + str(tree.value) + " </" + tree.name + ">")

        return temp

    def createXMLMinimized(self, tree):
        temp = []
        if str(type(tree)) == "<class '__main__.NTerminalNode'>":

            if len(tree.children) != 1 or tree.name == "PROGRAM":
                temp.append(self.space + "<" + tree.name + ">")

                self.stack.append(tree.name); self.space += "  "

                for child in tree.children:
                    temp.append(self.createXMLMinimized(child))
                    temp = self.flatten(temp)
                
                self.space = self.space[:-2]
                temp.append(self.space + "</" + self.stack[-1] + ">")

                self.stack.pop(); 
            else:
                for child in tree.children:
                    temp.append(self.createXMLMinimized(child))
                    temp = self.flatten(temp)
                    
        else:
            temp.append(self.space + "<" + str(tree.name) + "> " + str(tree.value) + " </" + tree.name + ">")

        return temp

    def flatten(self, temp):
        flattened = []
        for item in temp:
            if isinstance(item, list):
                flattened.extend(self.flatten(item))
            else:
                flattened.append(item)
        return flattened

    # SAVE AST IN XML FORMAT
    def printXML(self, xmlFull, xmlMin):
        f1 = open("TextFiles/xmlFull.txt", "w"); f1.write(""); f1.close()
        f2 = open("TextFiles/xmlFull.txt", "a")
        for line in xmlFull:
            f2.write(line); f2.write("\n")
        f2.close()

        f1 = open("TextFiles/xmlMinimized.txt", "w"); f1.write(""); f1.close()
        f2 = open("TextFiles/xmlMinimized.txt", "a")
        for line in xmlMin:
            f2.write(line); f2.write("\n")
        f2.close()



# Task 4 - Semantic Analysis Pass


In [274]:
# SYMBOL TABLE CLASS
class symbolTable:
    def __init__(self):
        self.variables = []
        self.functions = []


# SEMANTIC ANALYSER CLASS
class semanticAnalysis:
    def __init__(self, tree):
        self.symbols = symbolTable()
        self.tree = tree

        print("\nS E M A N T I C   A N A L Y S I S")
        self.traverse(self.tree)
        print("Code passed Semantic Analysis")

    # TRAVERSING THE AST
    def traverse(self, tree):
        if tree.name == "ASSIGNMENT":
            vName = tree.children[0].value
            
            flag = False
            for scope in self.symbols.variables:
                if vName in scope:
                    flag = True
                    break

            if flag == False:
                raise Exception ("Error: variable '" + vName + "' not defined")

            vValue = tree.children[2]
            vType = self.getVarType(vValue)

            reqType = ""
            for scope in self.symbols.variables:
                if vName in scope:
                    reqType = scope[vName]

            if vType != reqType and vType != None:
                raise Exception ("Error: variable '" + vName + "' does not accept type '" + vType + "'")   

        if tree.name == "VARIABLEDECL":
            vName = tree.children[0].value
            vType = tree.children[1].value

            for scope in self.symbols.variables:
                if vName in scope:
                    raise Exception ("Error: variable '" + vName + "' already declared")
                
            vValue = tree.children[3]
            vValueType = self.getVarType(vValue)

            if vType != vValueType:
                raise Exception ("Error: variable '" + vName + "' does not accept type '" + str(vValueType) + "'")
                
            self.symbols.variables[-1][vName] = vType

        if tree.name == "PIXELSTATEMENT":        
            if tree.children[0].name == "__PIXEL":
                a = self.getVarType(tree.children[1])
                b = self.getVarType(tree.children[2])
                c = self.getVarType(tree.children[3])

                if a == "int" and b == "int" and (c == "colour" or c == "int"):
                    pass
                else:
                    raise Exception ("Error: invalid format for __PIXEL (int, int, colour)")
            else:
                a = self.getVarType(tree.children[1])
                b = self.getVarType(tree.children[2])
                c = self.getVarType(tree.children[3])
                d = self.getVarType(tree.children[4])
                e = self.getVarType(tree.children[5])

                if a == "int" and b == "int" and c == "int" and d == "int" and (e == "colour" or e == "int"):
                    pass
                else:
                    raise Exception ("Error: invalid format for __PIXELR (int, int, int, int, colour)")
            
        if tree.name == "IFSTATEMENT":
            self.symbols.variables.append({}); self.symbols.functions.append({})
            for i in tree.children:
                self.traverse(i)   
            self.symbols.variables.pop(-1); self.symbols.functions.pop(-1)    

        if tree.name == "FORSTATEMENT":
            self.symbols.variables.append({}); self.symbols.functions.append({})
            for i in tree.children:
                self.traverse(i)   
 
            self.symbols.variables.pop(-1); self.symbols.functions.pop(-1)    

        if tree.name == "WHILESTATEMENT":
            self.symbols.variables.append({}); self.symbols.functions.append({})
            for i in tree.children:
                self.traverse(i)
            self.symbols.variables.pop(-1); self.symbols.functions.pop(-1)

        if tree.name == "FUNCTIONDECL":
            funName = tree.children[0].value
            funType = ""
            
            # No Parameters
            if tree.children[1].name == "ARROW":
                funType = tree.children[2].value
                for scope in self.symbols.functions:
                    if funName in scope:
                        raise Exception ("Error: function '" + funName + "' already declared")                
                self.symbols.functions[-1][funName] = [funType, None]

                self.symbols.variables.append({}); self.symbols.functions.append({})
                self.traverse(tree.children[3])

                listOfReturns = self.getReturns(tree)

                temp = []
                for x in listOfReturns:
                    temp.append(self.getVarType(x.children[0]))
                    
                flag = True
                for x in temp:
                    if x != funType:
                        flag = False

                if flag == False:
                    raise Exception ("Error: Return values dont match function type")
                
                self.symbols.variables.pop(-1); self.symbols.functions.pop(-1)
                
            # Yes Parameters
            else:
                funType = tree.children[3].value

                for scope in self.symbols.functions:
                    if funName in scope:
                        raise Exception ("Error: function '" + funName + "' already declared")

                list = []

                for param in tree.children[1].children:
                    pName = param.children[0].value

                    for scope in self.symbols.variables:
                        if pName in scope:
                            raise Exception ("Error: variable '" + pName + "' already declared")
                    
                    pType = param.children[1].value
                    list.append((pName, pType))

                parameterTypes = []
                for i in list:
                    parameterTypes.append(i[1])

                self.symbols.functions[-1][funName] = [funType, parameterTypes]

                self.symbols.variables.append({}); self.symbols.functions.append({})
                
                for pair in list:
                    self.symbols.variables[-1][pair[0]] = pair[1]

                self.traverse(tree.children[4])

                listOfReturns = self.getReturns(tree)

                temp = []
                for x in listOfReturns:
                    temp.append(self.getVarType(x.children[0]))

                flag = True
                for x in temp:
                    if x != funType:
                        flag = False

                if flag == False:
                    raise Exception ("Error: Return values dont match function type")
                
                self.symbols.variables.pop(-1); self.symbols.functions.pop(-1)

        if tree.name == "STATEMENT":
            self.traverse(tree.children[0])

        if tree.name == "BLOCK":
            for child in tree.children:
                self.traverse(child)

        if tree.name == "PROGRAM":
            self.symbols.variables.append({}); self.symbols.functions.append({})
            for child in tree.children:
                self.traverse(child)
            self.symbols.variables.pop(-1); self.symbols.functions.pop(-1)
    
    # GETTING VALUE TYPE OF EXPRESSION
    def getVarType(self, tree):        

        if str(type(tree)) == "<class '__main__.NTerminalNode'>":
            if tree.name == "FUNCTIONCALL":
                funName = tree.children[0].value
                reqParams = ""
                for scope in self.symbols.functions:
                    if funName in scope:
                        reqParams = scope[funName][1]

                if reqParams == "":
                    raise Exception ("Error: function '"+ funName + "' isnt declared")
                
                if reqParams == None:
                    if len(tree.children) != 1:
                        raise Exception ("Error: function '" + funName + "' doesnt take paramters")

                else:
                    if not tree.children[1] or len(tree.children[1].children) != len(reqParams):
                        raise Exception ("Error: more values than expected")

                    tempList = []
                    for i in tree.children[1].children:
                        tempList.append(self.getVarType(i))

                    for counter in range(len(tempList)):
                        if tempList[counter] != reqParams[counter]:
                            raise Exception ("Error: invalid paramters passed")
                        
                for scope in self.symbols.functions:
                    if funName in scope:
                        return scope[funName][0]

            else:                  
                types = []
                if tree.name == "EXPR":
                    for i in tree.children:
                        if i.name == "RELATIONALOP":
                            return "bool"
                        
                if tree.name == "TERM":
                    for i in tree.children:
                        if i.name == "MULTIPLICATIVEOP" and i.value == "/":
                            return "float"
                
                for child in tree.children:
                    varType = self.getVarType(child)

                    if varType != None:
                        types.append(varType)

                if len(types) == 1:
                    return types[0]
                else:
                    return self.checkTypes(types)

        else:
            if tree.name == "INTEGERLITERAL" or tree.name == "PADWIDTH" or tree.name == "PADHEIGHT":
                return ("int")
            
            elif tree.name == "FLOATLITERAL":
                return ("float")
            
            elif tree.name == "BOOLEANLITERAL":
                return ("bool")
            
            elif tree.name == "COLOURLITERAL":
                return ("colour")
            
            elif tree.name == "IDENTIFIER":
                name = tree.value
                reqType = ""

                for scope in self.symbols.variables:
                    if name in scope:
                        reqType = scope[name]

                if reqType == "":
                    raise Exception ("Error: variable '" + name + "' does not exist")

                return reqType

    def checkTypes(self, list):
        if len(list) == 0:
            return None

        elif len(list) == 1:           
            return list[0]

        elif len(list) > 2:
            while len(list) > 2:
                tempList = [list.pop(0), list.pop(0)]
                newVal = self.checkTypes(tempList)
                list.append(newVal)
            
        else:
            if list[0] == "int" and list[1] == "int":
                return "int"
            elif list[0] == "float" and list[1] == "int" or list[0] == "int" and list[1] == "float" or list[0] == "float" and list[1] == "float":
                return "float"
            elif list[0] == "colour" and list[1] == "colour":
                return "colour"
            elif list[0] == "bool" and list[1] == "bool":
                return "bool"
            else:
                raise Exception ("Error in type matching")
            
    # GETGET EACH RETURN TYPE IN A FUNCTION DECLERATION
    def getReturns(self, tree):
        num = []
        if tree.name == "RTRNSTATEMENT":
            num.append(tree)
            return num
        
        elif str(type(tree)) == "<class '__main__.NTerminalNode'>":

            for child in tree.children:
                temp = self.getReturns(child)
                
                for i in temp:
                    num.append(i)

        return num



# Task 5 - PixIR Code Generation Pass


In [275]:
# CODE GENERATOR CLASS
class codeGeneration:
    def __init__(self, tree):
        self.tree = tree
        self.functionsList = []
        self.symbols = symbolTable()

        self.P = []
        
        print("\nC O D E   G E N E R A T I O N")
        self.traverse(self.tree, None)

        for function in self.functionsList:
            for line in function:
                self.P.append(line)

        self.printCode()
        print("PixIr code generated")

    # CREATE CODE
    def traverse(self, tree, list):        
        if tree.name == "ASSIGNMENT":
            varName = tree.children[0].value
            varValue = tree.children[2]
            
            if list == None:
                value = self.getVarValue(varValue)
                self.symbols.variables[-1][varName][0] = value

                self.addPush(self.symbols.variables[-1][varName][1][0])
                self.addPush(self.symbols.variables[-1][varName][1][1])
                self.P.append("st")
            
            else:
                value = self.getVarValueNP(varValue)

                if type(value) != type(list):
                    value = [value]
                value = self.flatten(value)

                for i in value:
                    list.append(i)

                loc = ()
                for i in self.symbols.variables:
                    if varName in i:
                        loc = i[varName][1]
                        break

                list.append("push " + str(loc[0]))
                list.append("push " + str(loc[1]))
                list.append("st")

        if tree.name == "VARIABLEDECL":
            if list == None:
                self.P.append("push 1")
                
                if len(self.symbols.variables[-1]) < 1:
                    self.P.append("oframe")
                else:
                    self.P.append("alloc")

                if "tempNameToRemove" in self.symbols.variables[-1]:
                    self.symbols.variables[-1].pop("tempNameToRemove")

                varName = tree.children[0].value
                varValue = tree.children[3]

                value = self.getVarValue(varValue)
                self.symbols.variables[-1][varName] = [value, ((len(self.symbols.variables[-1])), 0)]
                self.addPush(self.symbols.variables[-1][varName][1][0])
                self.addPush(self.symbols.variables[-1][varName][1][1])
                self.P.append("st")

            else:
                list.append("push 1")

                if len(self.symbols.variables[-1]) < 1:
                    list.append("oframe")
                else:
                    list.append("alloc")

                if "tempNameToRemove" in self.symbols.variables[-1]:
                    self.symbols.variables[-1].pop("tempNameToRemove")

                varName = tree.children[0].value
                varValue = tree.children[3]

                temp = self.getVarValueNP(varValue)

                if type(temp) != type(list):
                    temp = [temp]
                temp = self.flatten(temp)
                
                for i in temp:
                    list.append(i)
                
                self.symbols.variables[-1][varName] = [list[-1][5:], ((len(self.symbols.variables[-1])), 0)]
                list.append("push " + str(self.symbols.variables[-1][varName][1][0]))
                list.append("push " + str(self.symbols.variables[-1][varName][1][1]))
                list.append("st")

        if tree.name == "PRINTSTATEMENT":
            if list == None:
                self.getVarValue(tree.children[0])
                self.P.append("print")
            
            else:
                list.append(self.getVarValueNP(tree.children[0]))
                list.append("print")
                return list

        if tree.name == "DELAYSTATEMENT":
            if list == None:
                self.getVarValue(tree.children[0])
                self.P.append("delay")
            
            else:
                list.append(self.getVarValueNP(tree.children[0]))
                list.append("delay")

        if tree.name == "PIXELSTATEMENT":   
            if tree.children[0].name == "__PIXEL":
                
                if list == None:
                    self.getVarValue(tree.children[-1])
                
                    for i in tree.children[1:-1]:
                        self.getVarValue(i)
                    self.P.append("pixel")
                
                else:
                    list.append(self.getVarValueNP(tree.children[-1]))
                    
                    for i in tree.children[1:-1]:
                        list.append(self.getVarValueNP(i))
                    list.append("pixel")

            if tree.children[0].name == "__PIXELR":
                
                if list == None:
                    self.getVarValue(tree.children[-1])
                
                    for i in tree.children[1:-1]:
                        self.getVarValue(i)
                    self.P.append("pixelr")
               
                else:
                    list.append(self.getVarValueNP(tree.children[-1]))
                
                    for i in tree.children[1:-1]:
                        list.append(self.getVarValueNP(i))
                    list.append("pixelr")

        if tree.name == "RTRNSTATEMENT":
            temp = self.getVarValueNP(tree.children[0])
            
            if type(temp) != type(list):
                temp = [temp]
            temp = self.flatten(temp)
            
            for i in temp:
                list.append(i)
                
            list.append("ret")

        if tree.name == "IFSTATEMENT":
            tempList = []
            self.symbols.variables.append({}); self.symbols.functions.append({})

            for i in self.symbols.variables[:-1]:
                for j in i:
                    if j != "tempNameToRemove":
                        i[j][1] = (i[j][1][0], i[j][1][1]+1)


            tempList.append("push #PC+")
            count1_1 = len(tempList)-1

            tempList.append(self.getVarValueNP(tree.children[0]))
            tempList = self.flatten(tempList)
            
            tempList.append("cjmp")

            if len(tree.children) == 3:
                self.traverse(tree.children[2], tempList)
                tempList = self.flatten(tempList)
            
            tempList.append("push #PC+")
            count2_1 = len(tempList)-1

            tempList.append("jmp")
            count1_2 = len(tempList)
            
            self.symbols.variables[-1]["tempNameToRemove"] = [None, (None, None)]
            
            self.traverse(tree.children[1], tempList)
            if "tempNameToRemove" in self.symbols.variables[-1]:
                    self.symbols.variables[-1].pop("tempNameToRemove")
            tempList = self.flatten(tempList)
            

            count2_2 = len(tempList)

            values = [count1_2-count1_1, count2_2-count2_1]
            
            for i in range(len(tempList)):
                if tempList[i] == "push #PC+":
                    tempList[i] = tempList[i] + str(values[0])
                    values.pop(0)

            tempList = ["oframe"] + tempList

            if list == None:
                for i in tempList:
                    self.P.append(i)
            else:                                
                for i in tempList:
                    list.append(i)

            for i in self.symbols.variables[:-1]:
                for j in i:
                    if j != "tempNameToRemove":
                        i[j][1] = (i[j][1][0], i[j][1][1]-1)

            self.symbols.variables.pop(-1); self.symbols.functions.pop(-1)

            if list == None:
                self.P.append("cframe")
            else:
                list.append("cframe")

        if tree.name == "FORSTATEMENT":            
            tempList = []
            self.symbols.variables.append({}); self.symbols.functions.append({})

            for i in self.symbols.variables[:-1]:
                for j in i:
                    if j != "tempNameToRemove":
                        i[j][1] = (i[j][1][0], i[j][1][1]+1)

            if len(tree.children)-1 == 3:
                # VARDECL
                self.traverse(tree.children[0], tempList)
                tempList = self.flatten(tempList)

                tempList.append("push #PC+")
                count1_1 = len(tempList)-1
                count3_2 = len(tempList)-1


                # EXPR
                tempList.append(self.getVarValueNP(tree.children[1]))
                tempList = self.flatten(tempList)

                tempList.append("cjmp")

                tempList.append("push #PC+")
                count2_1 = len(tempList)-1
                tempList.append("jmp")
                count1_2 = len(tempList)


                # BLOCK
                self.symbols.variables[-1]["tempNameToRemove"] = [None, (None, None)]
                self.traverse(tree.children[3], tempList)
                if "tempNameToRemove" in self.symbols.variables[-1]:
                    self.symbols.variables[-1].pop("tempNameToRemove")
                tempList = self.flatten(tempList)

                
                # ASSIGNMENT
                self.traverse(tree.children[2], tempList)
                tempList = self.flatten(tempList)

                tempList.append("push #PC-")
                count3_1 = len(tempList)-1
                tempList.append("jmp")
                count2_2 = len(tempList)-1

                values = [count1_2-count1_1, count2_2-count2_1+1, count3_1-count3_2]

                for i in range(len(tempList)):
                    if tempList[i] == "push #PC+" or tempList[i] == "push #PC-":
                        tempList[i] = tempList[i] + str(values[0])
                        values.pop(0)

                if list == None:
                    for i in tempList:
                        self.P.append(i)
                else:
                    for i in tempList:
                        list.append(i)

            elif len(tree.children)-1 == 2:
                if tree.children[0].name == "EXPR":
                    tempList.append("push #PC+")
                    count1_1 = len(tempList)-1
                    count3_2 = len(tempList)-1

                    # EXPR
                    tempList.append(self.getVarValueNP(tree.children[0]))
                    tempList = self.flatten(tempList)

                    tempList.append("cjmp")

                    tempList.append("push #PC+")
                    count2_1 = len(tempList)-1
                    tempList.append("jmp")
                    count1_2 = len(tempList)

                    # BLOCK
                    self.symbols.variables[-1]["tempNameToRemove"] = [None, (None, None)]
                    self.traverse(tree.children[2], tempList)
                    if "tempNameToRemove" in self.symbols.variables[-1]:
                        self.symbols.variables[-1].pop("tempNameToRemove")
                    tempList = self.flatten(tempList)

                    # ASSIGNMENT
                    self.traverse(tree.children[1], tempList)
                    tempList = self.flatten(tempList)

                    tempList.append("push #PC-")
                    count3_1 = len(tempList)-1
                    tempList.append("jmp")
                    count2_2 = len(tempList)-1

                    values = [count1_2-count1_1, count2_2-count2_1+1, count3_1-count3_2]

                    for i in range(len(tempList)):
                        if tempList[i] == "push #PC+" or tempList[i] == "push #PC-":
                            tempList[i] = tempList[i] + str(values[0])
                            values.pop(0)

                    if list == None:
                        for i in tempList:
                            self.P.append(i)
                    else:
                        for i in tempList:
                            list.append(i)
                            
                else:
                    # VARDECL
                    self.traverse(tree.children[0], tempList)
                    tempList = self.flatten(tempList)

                    tempList.append("push #PC+")
                    count1_1 = len(tempList)-1
                    count3_2 = len(tempList)-1

                    # EXPR
                    tempList.append(self.getVarValueNP(tree.children[1]))
                    tempList = self.flatten(tempList)

                    tempList.append("cjmp")

                    tempList.append("push #PC+")
                    count2_1 = len(tempList)-1
                    tempList.append("jmp")
                    count1_2 = len(tempList)

                    # BLOCK
                    self.symbols.variables[-1]["tempNameToRemove"] = [None, (None, None)]
                    self.traverse(tree.children[2], tempList)
                    if "tempNameToRemove" in self.symbols.variables[-1]:
                        self.symbols.variables[-1].pop("tempNameToRemove")
                    tempList = self.flatten(tempList)

                    tempList.append("push #PC-")
                    count3_1 = len(tempList)-1
                    tempList.append("jmp")
                    count2_2 = len(tempList)-1

                    values = [count1_2-count1_1, count2_2-count2_1+1, count3_1-count3_2]

                    for i in range(len(tempList)):
                        if tempList[i] == "push #PC+" or tempList[i] == "push #PC-":
                            tempList[i] = tempList[i] + str(values[0])
                            values.pop(0)

                    if list == None:
                        for i in tempList:
                            self.P.append(i)
                    else:
                        for i in tempList:
                            list.append(i)
            
            else:
                tempList.append("push #PC+")
                count1_1 = len(tempList)-1
                count3_2 = len(tempList)-1

                # EXPR
                tempList.append(self.getVarValueNP(tree.children[0]))
                tempList = self.flatten(tempList)

                tempList.append("cjmp")

                tempList.append("push #PC+")
                count2_1 = len(tempList)-1
                tempList.append("jmp")
                count1_2 = len(tempList)

                # BLOCK
                self.symbols.variables[-1]["tempNameToRemove"] = [None, (None, None)]
                self.traverse(tree.children[1], tempList)
                if "tempNameToRemove" in self.symbols.variables[-1]:
                        self.symbols.variables[-1].pop("tempNameToRemove")
                tempList = self.flatten(tempList)

                tempList.append("push #PC-")
                count3_1 = len(tempList)-1
                tempList.append("jmp")
                count2_2 = len(tempList)-1

                values = [count1_2-count1_1, count2_2-count2_1+1, count3_1-count3_2]

                for i in range(len(tempList)):
                    if tempList[i] == "push #PC+" or tempList[i] == "push #PC-":
                        tempList[i] = tempList[i] + str(values[0])
                        values.pop(0)

                if list == None:
                    for i in tempList:
                        self.P.append(i)
                else:
                    for i in tempList:
                        list.append(i)

            for i in self.symbols.variables[:-1]:
                for j in i:
                    if j != "tempNameToRemove":
                        i[j][1] = (i[j][1][0], i[j][1][1]-1)

            self.symbols.variables.pop(-1); self.symbols.functions.pop(-1)

            if list == None:
                self.P.append("cframe")
            else:
                list.append("cframe")

        if tree.name == "WHILESTATEMENT":
            tempList = []
            self.symbols.variables.append({}); self.symbols.functions.append({})

            for i in self.symbols.variables[:-1]:
                for j in i:
                    if j != "tempNameToRemove":
                        i[j][1] = (i[j][1][0], i[j][1][1]+1)
            
            tempList.append("push #PC+")
            count1_1 = len(tempList)-1

            tempList.append(self.getVarValueNP(tree.children[0]))
            tempList = self.flatten(tempList)

            tempList.append("cjmp")

            tempList.append("push #PC+")
            count2_1 = len(tempList)-1
            tempList.append("jmp")

            count1_2 = len(tempList)

            for i in tree.children[1].children:
                self.getVarValueNP(i)


            self.symbols.variables[-1]["tempNameToRemove"] = [None, (None, None)]
            self.traverse(tree.children[1], tempList)
            if "tempNameToRemove" in self.symbols.variables[-1]:
                self.symbols.variables[-1].pop("tempNameToRemove")
            tempList = self.flatten(tempList)


            tempList.append("push #PC-")
            count3_1 = len(tempList)-1
            tempList.append("jmp")
            count2_2 = len(tempList)-1

            values = [count1_2-count1_1, count2_2-count2_1+1, count3_1]

            for i in range(len(tempList)):
                if tempList[i] == "push #PC+" or tempList[i] == "push #PC-":
                    tempList[i] = tempList[i] + str(values[0])
                    values.pop(0)

            tempList = ["oframe"] + tempList

            if list == None:
                for i in tempList:
                    self.P.append(i)
            else:
                for i in tempList:
                    list.append(i)

            for i in self.symbols.variables[:-1]:
                for j in i:
                    if j != "tempNameToRemove":
                        i[j][1] = (i[j][1][0], i[j][1][1]-1)

            self.symbols.variables.pop(-1); self.symbols.functions.pop(-1)

            if list == None:
                self.P.append("cframe")
            else:
                list.append("cframe")

        if tree.name == "FUNCTIONDECL":
            list = []
            functionCode = []
            functionCode.append("." + tree.children[0].value)

            # no params
            if tree.children[1].name == "ARROW":
                self.traverse(tree.children[3], functionCode)
            #yes params
            else:
                params = []

                for i in tree.children[1].children:
                    params = [i] + params

                block = tree.children[4]

                self.symbols.variables.append({}); self.symbols.functions.append({})
            
                if params != None:
                    for param in params:
                        vName = param.children[0].value
                        self.symbols.variables[-1][vName] = ["0", ((len(self.symbols.variables[-1])), 0)]

                for child in block.children:
                    self.traverse(child, functionCode)

                self.symbols.variables.pop(-1); self.symbols.functions.pop(-1)

            self.functionsList.append(functionCode)

        if tree.name == "STATEMENT":
            if list == None:
                self.traverse(tree.children[0], None)
            else:
                return self.traverse(tree.children[0], list)

        if tree.name == "BLOCK":
            for child in tree.children:
                self.traverse(child, list)

        if tree.name == "PROGRAM":
            self.P.append(".main")
            self.symbols.variables.append({}); self.symbols.functions.append({})
            
            for child in tree.children:
                self.traverse(child, None)
            
            self.symbols.variables.pop(-1); self.symbols.functions.pop(-1)
            self.P.append("halt")

    def addPush(self, toPush):
        if toPush in ["height", "width"]:
            self.P.append(toPush)
        elif type(toPush) == type(tuple()):
            self.P.append("push [" + str(toPush[0]) + ":" + str(toPush[1]) + "]")
        else:
            self.P.append("push " + str(toPush))

    def getVarValue(self, value):

        if value.name == "PADRANDI":
            self.getVarValue(value.children[1])
            
            self.P.append("irnd")

        if value.name == "FUNCTIONCALL":
            if len(value.children) == 1:
                self.P.append("push 0")
                self.P.append("push ." + value.children[0].value)
                self.P.append("call")
            else:  
                aParms = value.children[1].children
                for i in aParms:
                    self.getVarValue(i)

                self.P.append("push " + str(len(aParms)))
                self.P.append("push ." + value.children[0].value)
                self.P.append("call")

        if value.name == "SUBEXPR":
            self.getVarValue(value.children[0])

        if value.name == "EXPR":
            if len(value.children) > 1:
                counter = 0
                
                while counter < len(value.children)-1:
                    self.getVarValue(value.children[counter + 2])

                    if counter == 0:
                        temp = self.getVarValue(value.children[counter])

                    op = value.children[counter + 1].value
                    if op == "<": self.P.append("lt")
                    elif op == "<=": self.P.append("le")
                    elif op == ">": self.P.append("gt")
                    elif op == ">=": self.P.append("ge")
                    elif op == "==": self.P.append("eq")

                    counter += 2
                return temp
                                
            else:
                return self.getVarValue(value.children[0])

        if value.name == "SIMPLEEXPR":
            if len(value.children) > 1:
                counter = 0
                
                while counter < len(value.children)-1:
                    self.getVarValue(value.children[counter + 2])

                    if counter == 0:
                        temp = self.getVarValue(value.children[counter])

                    op = value.children[counter + 1].value
                    if op == "+": self.P.append("add")
                    elif op == "-": self.P.append("sub")

                    counter += 2
                return temp
                            
            else:
                return self.getVarValue(value.children[0])

        if value.name == "TERM":
            if len(value.children) > 1:
                counter = 0
                
                while counter < len(value.children)-1:
                    self.getVarValue(value.children[counter + 2])

                    if counter == 0:
                        temp = self.getVarValue(value.children[counter])

                    op = value.children[counter + 1].value
                    if op == "*": self.P.append("mul")
                    elif op == "/": self.P.append("div")

                    counter += 2
                return temp
            
            else:
                return self.getVarValue(value.children[0])

        if value.name == "FACTOR":
            return self.getVarValue(value.children[0])

        if value.name == "LITERAL":
            return self.getVarValue(value.children[0])

        if value.name == "BOOLEANLITERAL":
            self.addPush( value.value)
            return value.value

        if value.name == "INTEGERLITERAL":
            self.addPush(value.value)
            return value.value
        
        if value.name == "FLOATLITERAL":
            self.addPush(value.value)            
            return value.value
        
        if value.name == "COLOURLITERAL":
            self.addPush(value.value)            
            return value.value

        if value.name == "IDENTIFIER":
            for scope in self.symbols.variables:
                if value.value in scope:
                    self.addPush(scope[value.value][1])
                    return scope[value.value][0]

        if value.name == "PADHEIGHT":
            self.P.append("height")
            return 36
        
        if value.name == "PADWIDTH":
            self.P.append("width")
            return 36

    def getVarValueNP(self, value):
        tempMain = []

        if value.name == "PADRANDI":
            tempMain.append(self.getVarValueNP(value.children[1]))
            tempMain.append("irnd")
            return tempMain

        if value.name == "FUNCTIONCALL":
            if len(value.children) == 1:
                tempMain.append("push 0")
                tempMain.append("push ." + value.children[0].value)
                tempMain.append("call")
                return tempMain
    
            else:  
                aParms = value.children[1].children
                for i in aParms:
                    tempMain.append(self.getVarValueNP(i))

                tempMain.append("push " + str(len(aParms)))
                tempMain.append("push ." + value.children[0].value)
                tempMain.append("call")
                return tempMain

        if value.name == "SUBEXPR":
            return self.getVarValueNP(value.children[0])

        if value.name == "EXPR":
            if len(value.children) > 1:
                counter = 0
                
                while counter < len(value.children)-1:
                    temp1 = self.getVarValueNP(value.children[counter + 2])
                    tempMain.append(temp1)

                    if counter == 0:
                        temp = self.getVarValueNP(value.children[counter])
                        tempMain.append(temp)

                    op = value.children[counter + 1].value
                    if op == "<": tempMain.append("lt")
                    elif op == "<=": tempMain.append("le")
                    elif op == ">": tempMain.append("gt")
                    elif op == ">=": tempMain.append("ge")
                    elif op == "==": tempMain.append("eq")

                    counter += 2
                return tempMain
                                
            else:
                return self.getVarValueNP(value.children[0])

        if value.name == "SIMPLEEXPR":
            if len(value.children) > 1:
                counter = 0
                
                while counter < len(value.children)-1:
                    temp1 = self.getVarValueNP(value.children[counter + 2])
                    tempMain.append(temp1)

                    if counter == 0:
                        temp = self.getVarValueNP(value.children[counter])
                        tempMain.append(temp)

                    op = value.children[counter + 1].value
                    if op == "+": tempMain.append("add")
                    elif op == "-": tempMain.append("sub")

                    counter += 2
                return tempMain
                            
            else:
                return self.getVarValueNP(value.children[0])

        if value.name == "TERM":
            if len(value.children) > 1:
                counter = 0
                
                while counter < len(value.children)-1:
                    temp1 = self.getVarValueNP(value.children[counter + 2])
                    tempMain.append(temp1)

                    if counter == 0:
                        temp = self.getVarValueNP(value.children[counter])
                        tempMain.append(temp)

                    op = value.children[counter + 1].value
                    if op == "*": tempMain.append("mul")
                    elif op == "/": tempMain.append("div")

                    counter += 2
                return tempMain
            else:
                return self.getVarValueNP(value.children[0])

        if value.name == "FACTOR":
            return self.getVarValueNP(value.children[0])
    
        if value.name == "LITERAL":
            return self.getVarValueNP(value.children[0])

        if value.name == "BOOLEANLITERAL":
            return "push " + value.value

        if value.name == "INTEGERLITERAL":
            return "push " + value.value
        
        if value.name == "FLOATLITERAL":
            return "push " + value.value
        
        if value.name == "COLOURLITERAL":
           return "push " + value.value

        if value.name == "IDENTIFIER":
            for scope in self.symbols.variables:
                if value.value in scope:
                    locations = (scope[value.value][1])
                    return ("push [" + str(locations[0]) + ":" + str(locations[1]) + "]")

        if value.name == "PADHEIGHT":
            return "height"
        
        if value.name == "PADWIDTH":
            return "width"
        
    def flatten(self, temp):
        flattened = []
        for item in temp:
            if isinstance(item, list):
                flattened.extend(self.flatten(item))
            else:
                flattened.append(item)
        return flattened  

    # PRINT/SAVE CODE
    def printCode(self):
        f = open("TextFiles/pixIrCode.txt", "w"); f.write(""); f.close()

        f = open("TextFiles/pixIrCode.txt", "w")
        self.P = self.flatten(self.P)
        for line in self.P:
            f.write(line); f.write("\n")
            if line == "halt" or line == "ret":
                f.write("\n")
        f.close()



---

In [276]:
programs = ["TextFiles/inputPrograms/program1.txt", "TextFiles/inputPrograms/program2.txt", 
            "TextFiles/inputPrograms/program3.txt", "TextFiles/inputPrograms/program4.txt"]

f = open(programs[3], 'r'); code = f.read(); f.close()
lexer = Lexer(code)
parser = Parser(lexer.tokenList)
xml = XMLGeneration(parser.tree)
semAnalysis = semanticAnalysis(parser.tree)
codeGen = codeGeneration(parser.tree)

L E X E R
Code accepted by Lexer

P A R S E R
Code Accepted by Parser

X M L   G E N E R A T I O N
Printed XML Full and XML Minimized

S E M A N T I C   A N A L Y S I S
Code passed Semantic Analysis

C O D E   G E N E R A T I O N
PixIr code generated
